# Linear regression

## Algorithm

In this notebook we implement from scratch the simple learning algorithm linear regression. The idea is easy. We have some data X of size (m,n) where each row is a data point characterized by n features. We also have the targets associated with each example in the form of a (m,1) vector. We want to model this data with a hypothesis of the form

$$ h(X) = \beta_0 + X\beta $$

It is simpler to add an extra feature for the parameter $\beta_0$ by including a column of ones into X. The hypothesis then looks like

$$ h(X) = X\beta $$

The goal is to try to minimize the difference between the predictions and the actual data, which is computed using the cost function

$$ J(\beta) = (Y-X\beta)^T(Y-X\beta) $$

There are two ways of finding the best set of parameters $\beta$ to describe the data:

- Gradient descent: Initialize the weights to 0 and iteratively move it along the gradient of the cost function. The algorithm consists of repeating the following step many times

$$ \beta = \beta - \alpha\frac{\partial J}{\partial\beta} $$

The parameter $\alpha$ is the learning rate and the gradient is given by

$$ \frac{\partial J}{\partial\beta} = -2X^T(Y-X\beta) $$

We iterate like this until the cost stops decreasing.

- There is actually an exact solution to this optimization problem, called the normal equation. The optimal weights can be found to be

$$ \beta = (X^T X)^{-1}X^T Y $$

## Regularization

To prevent overfitting it is useful to introduce regularization, which simply amounts to adding a term to the minimization problem. The two most common schemes used are:

- L1: Minimize the cost function $J(\beta) + \lambda\sum_{i=1}^n|\beta_i|$. It's important to not regularize the bias $\beta_0$. This is implemented by adding a term to the gradient descent of all the weights except $\beta_0$

$$ \beta_i = \beta_i - \alpha \frac{\partial J}{\partial\beta_i} - \lambda\,sign(\beta_i) $$

- L2: Minimize the cost function $J(\beta) + \lambda\sum_{i=1}^n\beta_i^2$. The new gradient descent algorithm is

$$ \beta_i = \beta_i - \alpha \frac{\partial J}{\partial\beta_i} - \lambda\beta_i $$

## Error metrics

To assess the success of the prediction two metrics are the most common to use. They are both based on the cost function, which is also called residual sum of squares (RSS):
    
- Residual standard error (RSE): This is basically the standard deviation between the prediction and the data and it is calculated from

$$ RSE = \sqrt{\frac{RSS}{m-2}} $$

It is an absolute estimation of the error and we want it to be as small as possible.

- $R^2$: This is an index that indicates the proportion of the variance in the data that is captured by the model. This is based off the actual variance of the data, called True sum of squared (TSS)

$$ TSS = (Y - \bar{Y})^T(Y - \bar{Y}) $$

The coefficient is computed by

$$ R^2 = 1-\frac{RSS}{TSS} $$

and it takes values between 0 and 1. The higher it is the better the model is.

## Feature engineering

There are two steps that may be useful to do before training the model:

- Normalization: It is important that all the features are on the same scale. This can be achieved by taking the training data and transforming each feature according to

$$ X_i \rightarrow \frac{X_i - \mu_i}{\sigma_i} $$

with $\mu_i$ the mean of the feature and $\sigma_i$ the standard deviation. This should put all the features mostly on a scale from -1 to 1. Note that the same conversion, using the training data, must be used when evaluating the model on new data.

- Polynomial: We can use the features and combine them into all the possible multiples of each other up to a certain degree. This will generate a model that has more predicting power since it doesn't just assume that the data is linear but it uses a more general hypothesis. This is called polynomial regression. It however increases quickly the number of features so it may lead to overfitting.

# Code

In [1]:
import numpy as np
from numpy.linalg import inv
import itertools

In [158]:
class lin_reg:
    '''Model that does linear regression on the data. There is also the option of transforming the data
    to do polynomial regression.'''
    
    def __init__(self, degree = None, normalize = False):
        '''Set parameters to decide if we normalize the data and if we create polynomial features.'''
        if degree >=2:
            self.degree = degree
        else:
            self.degree = None
        if normalize:
            self.normalize = True
    
    def train(self, X, Y, alpha = 0.0001, limit = 100000, normal = False, lambd = 0, regularization = None):
        '''Train the model on a dataset from the examples X and the labels Y.
        The algorithm uses gradient descent with learning rate 'alpha' for a max of 'limit' steps.
        X must in the format (# examples)x(# features) and Y is a column array.
        The final weights as well as a list of the costs calculated through the training 
        can be extracted using methods .weights and .cost.
        Option to use the exact solution using "normal" instead of gradient descent.
        There are the option of including L1 or L2 regularization.'''
        
        # Normalize the data and save the normalization parameters
        if self.normalize:
            self.mean = X.mean(axis=0)
            self.std = X.std(axis=0)
            X = (X - self.mean) / self.std
        # Transform data to polynomial features if asked to
        if self.degree != None:
            X = self.polynomial(X, self.degree)
        # Extract number of training examples and features from X
        (m, n) = X.shape
        # Add a column of ones to X for the bias
        X = np.append(np.ones((m,1)), X, axis = 1)
        Y = np.reshape(Y, (m,1))
            
        if not normal:
            # Initialize the weights to zero
            self.weights = np.zeros((n+1 ,1))
            # Initialize the cost
            i = 0
            self.cost = []
            self.cost.append(np.matmul(np.transpose(Y - np.matmul(X, self.weights)), Y - np.matmul(X, self.weights)))
            # Update weights with gradient and compute new cost
            while True:
                self.weights = self.weights - alpha * self.gradient(X, Y, self.weights, lambd, regularization)
                self.cost.append(np.matmul(np.transpose(Y - np.matmul(X, self.weights)), Y - np.matmul(X, self.weights)))
                # Stop upgrading if cost doesn't lower of if reached limit
                if self.cost[i + 1] < self.cost[i]:
                    if i % 10000 == 0:
                        print(i, 'steps done', end="\r")
                    i += 1
                else:
                    break
                if i > limit:
                    print('Reached the limit')
                    break
        else:
            self.weights = np.matmul(np.matmul(inv(np.matmul(np.transpose(X), X)), np.transpose(X)), Y)

            
    def gradient(self, X, Y, weights, lambd, regularization):
        '''Compute the gradient involved in gradient descent, for different regularization schemes.'''
        
        (m, n) = X.shape
        basic_grad = -2 * np.matmul(np.transpose(X), Y - np.matmul(X, weights))
        if regularization == 'L1':
            return basic_grad + lambd * np.insert(np.sign(weights[1:]), 0, 0).reshape(n, 1) 
        elif regularization == 'L2':
            return basic_grad + lambd * np.insert(weights[1:], 0, 0).reshape(n, 1)
        else:
            return basic_grad
    
    
    def predict(self, X):
        '''Use data X and trained model to predict labels using linear regression hypothesis.
        Output is an array with one value for each data point.'''
    
        # Normalize the data if asked to
        if self.normalize:
            X = (X - self.mean) / self.std
        # Transform data to polynomial features if asked to
        if self.degree != None:
            X = self.polynomial(X, self.degree)
        # Extract number of training examples and features from X
        (m, n) = X.shape
        # Add a column of ones to X for the bias
        X = np.append(np.ones((m,1)), X, axis = 1)
        return np.matmul(X, self.weights)
    
    
    def error(self, Y, Y_pred, metric):
        '''Computes the error of Y_pred compared to true answer Y.
        Option of using the R2 or RSE as metrics.'''
        
        # Reshape data and use it to make a prediction
        m = Y.shape[0]
        Y = np.reshape(Y, (m,1))
        # Compute total squared error for use later
        rss = np.matmul(np.transpose(Y - Y_pred), Y - Y_pred)
        if metric == 'rse':
            rse_error = np.sqrt(rss/(m-2))
            print('RSE error is: ', rse_error)
        elif metric == 'r2':
            tss = np.matmul(np.transpose(Y - np.mean(Y)), Y - np.mean(Y))
            r2_error = 1 - rss/tss
            print('R squared error is: ', r2_error)
        else:
            print('Wrong metric specification!')
        
    def polynomial(self, X, degree):
        '''Combines the features into all possible combinations to form a polynomial of a given degree.<
        This should be used before doing anything else and on all the data.'''
        
        (m,n) = X.shape
        for deg in range(2, degree+1):
            for combin in itertools.combinations_with_replacement(range(n), deg):
                X = np.append(X, np.prod(X[:, list(combin)], axis=1).reshape(m,1), axis=1)
        return X
    

Future: add regularization, polynomial regression, visualization cost, normalize data

Notes: normalize before poly, can't use normal equation for poly, need very small alpha (not sure if ok)